In [14]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

In [15]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '3169d387b687441e28a8d7c2c6781875', 'Date': 'Tue, 20 Aug 2024 02:37:17 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [16]:
data = json.load(open("reviews.json"))
# data['reviews']

In [17]:
processed_data = []
client = OpenAI()

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )


In [18]:
processed_data[0]

{'values': [-0.02280946,
  0.00081052433,
  0.024379773,
  -0.0076452945,
  0.038246434,
  -0.009401916,
  -0.0041619944,
  0.047136005,
  0.012635696,
  -0.06723069,
  0.015237358,
  -0.03574458,
  0.017193595,
  -0.033242725,
  0.022729615,
  0.023647849,
  -0.028132556,
  -0.03529212,
  0.019588988,
  0.032790262,
  0.0476417,
  -0.01020038,
  0.04002967,
  -0.01687421,
  -0.019881759,
  -0.0041253986,
  0.033189494,
  0.04737554,
  0.04492692,
  -0.018790524,
  0.0409346,
  -0.01733998,
  -0.015317205,
  -0.014252586,
  -0.06297221,
  0.036143813,
  -0.0044847075,
  0.058713734,
  0.015237358,
  -0.020533837,
  0.029702868,
  -0.00758541,
  -0.043888915,
  0.00049612904,
  0.039470747,
  -0.03521227,
  -0.013667045,
  -0.030128716,
  0.01061292,
  0.033987958,
  -0.05956543,
  0.075534716,
  0.041387063,
  -0.045006767,
  -0.022796152,
  0.048440162,
  0.005975174,
  0.025244776,
  0.034413807,
  -0.027946247,
  0.08751167,
  -0.005073575,
  -0.03787382,
  -0.034892887,
  -0.033934

In [19]:
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

In [20]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}